<a href="https://colab.research.google.com/github/Zenith1618/LLM/blob/main/News_Text_Summarization_using_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
!pip install transformers sentencepiece

In [2]:
import pandas as pd
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

# Load the Dataset


Dataset Link: https://www.kaggle.com/datasets/sunnysai12345/news-summary

In [3]:
data = pd.read_csv("news_summary.csv", encoding='iso-8859-1')
data.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [4]:
data['headlines'].iloc[0]

'Daman & Diu revokes mandatory Rakshabandhan in offices order'

In [5]:
data['text'].iloc[0]

'The Administration of Union Territory Daman and Diu has revoked its order that made it compulsory for women to tie rakhis to their male colleagues on the occasion of Rakshabandhan on August 7. The administration was forced to withdraw the decision within 24 hours of issuing the circular after it received flak from employees and was slammed on social media.'

In [6]:
data['ctext'].iloc[0]

'The Daman and Diu administration on Wednesday withdrew a circular that asked women staff to tie rakhis on male colleagues after the order triggered a backlash from employees and was ripped apart on social media.The union territory?s administration was forced to retreat within 24 hours of issuing the circular that made it compulsory for its staff to celebrate Rakshabandhan at workplace.?It has been decided to celebrate the festival of Rakshabandhan on August 7. In this connection, all offices/ departments shall remain open and celebrate the festival collectively at a suitable time wherein all the lady staff shall tie rakhis to their colleagues,? the order, issued on August 1 by Gurpreet Singh, deputy secretary (personnel), had said.To ensure that no one skipped office, an attendance report was to be sent to the government the next evening.The two notifications ? one mandating the celebration of Rakshabandhan (left) and the other withdrawing the mandate (right) ? were issued by the Dama

#Tokeniztion and Model Inference

In [7]:
model_name = 't5-base'
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
device

device(type='cuda')

In [10]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [12]:
text = data['ctext'].iloc[4]
text

'Hotels in Mumbai and other Indian cities are to train their staff to spot signs of sex trafficking such as frequent requests for bed linen changes or a "Do not disturb" sign left on the door for days on end. The group behind the initiative is also developing a mobile phone app - Rescue Me - which hotel staff can use to alert local police and senior anti-trafficking officers if they see suspicious behavior. "Hotels are breeding grounds for human trade," said Sanee Awsarmmel, chairman of the alumni group of Maharashtra State Institute of Hotel Management and Catering Technology. "(We) have hospitality professionals working in hotels across the country. We are committed to this cause."The initiative, spearheaded by the alumni group and backed by the Maharashtra state government, comes amid growing international recognition that hotels have a key role to play in fighting modern day slavery. MAHARASHTRA MAJOR DESTINATION FOR TRAFFICKED GIRLS Maharashtra, of which Mumbai is the capital, is 

In [13]:
before_summarize = text.strip().replace("\n","")
print("Before text summarization:")
before_summarize

Before text summarization:


'Hotels in Mumbai and other Indian cities are to train their staff to spot signs of sex trafficking such as frequent requests for bed linen changes or a "Do not disturb" sign left on the door for days on end. The group behind the initiative is also developing a mobile phone app - Rescue Me - which hotel staff can use to alert local police and senior anti-trafficking officers if they see suspicious behavior. "Hotels are breeding grounds for human trade," said Sanee Awsarmmel, chairman of the alumni group of Maharashtra State Institute of Hotel Management and Catering Technology. "(We) have hospitality professionals working in hotels across the country. We are committed to this cause."The initiative, spearheaded by the alumni group and backed by the Maharashtra state government, comes amid growing international recognition that hotels have a key role to play in fighting modern day slavery. MAHARASHTRA MAJOR DESTINATION FOR TRAFFICKED GIRLS Maharashtra, of which Mumbai is the capital, is 

In [14]:
tokenized_text = tokenizer.encode(before_summarize, return_tensors="pt")

In [15]:
model = model.to(device)
tokenized_text = tokenized_text.to(device)

In [16]:
summary_ids = model.generate(tokenized_text,
no_repeat_ngram_size=3,
min_length=40,
max_length=150,
early_stopping=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


In [17]:
summary_ids

tensor([[    0, 32099,     3,     9,  1156,   951,  1120,     3,    18, 22175,
          1212,     3,    18,    12,  5685,   415,  2095,    11,  1181,    18,
         14793,  4638,  1765,  5803,     3,    99,    79,   217, 21641,  3889,
             3,     5,     8,  6121,    19, 23887, 22248,    57,     8, 10644,
           563,    13, 16256, 12380,  1313,  1015,  2548,    13,  2282,  2159,
            11, 29326,  3669,     3,     5,     1]], device='cuda:0')

In [19]:
after_summarize = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [20]:
print("After text summarization:")
after_summarize

After text summarization:


'a mobile phone app - Rescue Me - to alert local police and anti-trafficking officers if they see suspicious behavior. the initiative is spearheaded by the alumni group of Maharashtra State Institute of Hotel Management and Catering Technology.'

In [21]:
print("Before:",len(before_summarize))
print("After:",len(after_summarize))

Before: 3249
After: 243


# Using Pipeline

In [22]:
from transformers import pipeline

In [23]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base")

In [24]:
context = """
Elon Musk, a visionary entrepreneur, is a name synonymous with groundbreaking innovations that transcend industries. Born in South Africa in 1971, Musk embarked on a journey that would lead him to revolutionize the realms of technology, space exploration, and sustainable energy.

One of Musk's most notable contributions lies in the electric vehicle sector. He co-founded Tesla Motors in 2003, aiming to make electric cars a mainstream reality. With Tesla's sleek designs, impressive performance, and advancements in battery technology, Musk has not only disrupted the automotive industry but also accelerated the world's transition to sustainable transportation.

SpaceX, another brainchild of Musk, is at the forefront of space exploration. Musk's audacious goal to make life multiplanetary has driven SpaceX to develop reusable rockets and drastically reduce the cost of space travel. The successful landing of Falcon rockets and the launch of the Falcon Heavy have reshaped the space industry, making it more accessible and economical.
"""

In [25]:
summary = summarizer(context, max_length=150, min_length=50, do_sample=True, top_k=50, top_p=0.95)

###Hyperparameter
do_sample=True: This parameter enables sampling during the generation process. When set to True, the model will use sampling to generate text, which can lead to more diverse outputs.

top_k=50: This parameter controls the sampling method by restricting the selection of tokens to the top k most likely tokens at each step. In this case, only the top 50 tokens are considered.

top_p=0.95: This parameter, often referred to as nucleus sampling or top-p sampling, sets a threshold probability for selecting tokens during sampling. Tokens with cumulative probabilities up to top_p are considered for sampling.

In [26]:
summary[0]['summary_text']

'born in 1971, Elon Musk is a name synonymous with groundbreaking innovations . he co-founded Tesla Motors in 2003, aiming to make electric cars a mainstream reality . SpaceX is at the forefront of space exploration .'

In [27]:
print("Before:",len(context))
print("After:",len(summary[0]['summary_text']))

Before: 1042
After: 216
